In [26]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [27]:
dt=pd.read_excel('OnlineRetail.xls')

In [28]:
dt.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [29]:
dt.Description.unique()

array(['WHITE HANGING HEART T-LIGHT HOLDER', 'WHITE METAL LANTERN',
       'CREAM CUPID HEARTS COAT HANGER', ...,
       'SWISS ROLL TOWEL PINK  SPOTS', 'GLASS CHALICE GREEN  LARGE ',
       'GLASS BEAD HOOP EARRINGS GREEN'], dtype=object)

In [30]:
dt.Description=dt.Description.str.strip()

In [31]:
dt.Description[:5]

0     WHITE HANGING HEART T-LIGHT HOLDER
1                    WHITE METAL LANTERN
2         CREAM CUPID HEARTS COAT HANGER
3    KNITTED UNION FLAG HOT WATER BOTTLE
4         RED WOOLLY HOTTIE WHITE HEART.
Name: Description, dtype: object

In [32]:
dt.dropna(axis=0, subset=['InvoiceNo'], inplace=True)

In [33]:
dt['InvoiceNo'] = dt['InvoiceNo'].astype('str')

In [34]:
dt.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [35]:
df=dt.copy()

In [36]:
df = df[~df['InvoiceNo'].str.contains('C')]

In [37]:
data=df[df.Country=='France'].groupby(['InvoiceNo', 'Description'])['Quantity'].sum()

In [38]:
data[:5]

InvoiceNo  Description                    
536370     ALARM CLOCK BAKELIKE GREEN         12
           ALARM CLOCK BAKELIKE PINK          24
           ALARM CLOCK BAKELIKE RED           24
           CHARLOTTE BAG DOLLY GIRL DESIGN    20
           CIRCUS PARADE LUNCH BOX            24
Name: Quantity, dtype: int64

In [39]:
data=data.unstack().reset_index().fillna('0')

In [40]:
data.set_index('InvoiceNo').head()

Description,10 COLOUR SPACEBOY PEN,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE RED RETROSPOT,3 HOOK HANGER MAGIC GARDEN,3 PIECE SPACEBOY COOKIE CUTTER SET,4 TRADITIONAL SPINNING TOPS,5 HOOK HANGER MAGIC TOADSTOOL,5 HOOK HANGER RED MAGIC TOADSTOOL,6 RIBBONS EMPIRE,...,WOODLAND HEIGHT CHART STICKERS,WOODLAND STICKERS,WOODLAND CHARLOTTE BAG,WOODLAND DESIGN COTTON TOTE BAG,WOODLAND PARTY BAG + STICKER SET,WORLD WAR 2 GLIDERS ASSTD DESIGNS,WRAP GREEN PEARS,WRAP I LOVE LONDON,WRAP RED APPLES,ZINC METAL HEART DECORATION
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,12,0,...,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,...,0,12,10,0,0,0,0,0,0,0


In [41]:
#data.columns=data.columns.str.replace(' ','_')

In [42]:
for cl in data.columns:
    data[cl]=data[cl].astype(int)

In [43]:
def onehot(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
dataset=data.applymap(onehot)

In [44]:
dataset=dataset.set_index('InvoiceNo')

In [45]:
frequent_itemsets= apriori(dataset, min_support=0.1, use_colnames=True)

In [46]:
frequent_itemsets.head()

,support,itemsets
0,0.119048,(ALARM CLOCK BAKELIKE GREEN)
1,0.119048,(ALARM CLOCK BAKELIKE PINK)
2,0.119048,(ALARM CLOCK BAKELIKE RED)
3,0.119048,(ASSORTED COLOUR MINI CASES)
4,0.142857,(BAKING SET 9 PIECE RETROSPOT)


In [47]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.5)
rules.sort_values('lift',ascending=False)
rules.head(286)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.119048,0.119048,0.119048,1.000000,8.400000,0.104875,inf
1,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.119048,0.119048,0.119048,1.000000,8.400000,0.104875,inf
2,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.119048,0.119048,0.119048,1.000000,8.400000,0.104875,inf
3,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.119048,0.119048,0.119048,1.000000,8.400000,0.104875,inf
4,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE RED),0.119048,0.119048,0.119048,1.000000,8.400000,0.104875,inf
5,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE PINK),0.119048,0.119048,0.119048,1.000000,8.400000,0.104875,inf
6,(RED RETROSPOT MINI CASES),(ASSORTED COLOUR MINI CASES),0.309524,0.119048,0.119048,0.384615,3.230769,0.082200,1.431548
7,(ASSORTED COLOUR MINI CASES),(RED RETROSPOT MINI CASES),0.119048,0.309524,0.119048,1.000000,3.230769,0.082200,inf
8,(EDWARDIAN PARASOL RED),(RED RETROSPOT MINI CASES),0.119048,0.309524,0.119048,1.000000,3.230769,0.082200,inf
9,(RED RETROSPOT MINI CASES),(EDWARDIAN PARASOL RED),0.309524,0.119048,0.119048,0.384615,3.230769,0.082200,1.431548
